In [1]:
import FinanceDataReader as fdr
#비슷한 코드들에는 주석을 적지 않았습니다.

In [2]:
#krx에서 데이터를 가져옵니다.
stock_list = fdr.StockListing('KRX').dropna()

In [3]:
stock_list

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
0,060310,KOSDAQ,3S,전자부품 제조업,반도체 웨이퍼 캐리어,2002-04-23,03월,김세완,http://www.3sref.com,서울특별시
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,"박대현, 손삼달",http://www.ajnet.co.kr,서울특별시
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,1999-08-11,12월,"채형석, 이석주(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
3,054620,KOSDAQ,APS홀딩스,기타 금융업,인터넷 트래픽 솔루션,2001-12-04,12월,정기로,http://www.apsholdings.co.kr,경기도
4,265520,KOSDAQ,AP시스템,특수 목적용 기계 제조업,디스플레이 제조 장비,2017-04-07,12월,김영주,http://www.apsystems.co.kr,경기도
...,...,...,...,...,...,...,...,...,...,...
7849,189980,KOSDAQ,흥국에프엔비,비알코올음료 및 얼음 제조업,"과일음료(에이드베이스, 스무디, 착즙쥬스 등)",2015-08-07,12월,"박철범, 오길영",http://www.hyungkuk.com,서울특별시
7850,000540,KOSPI,흥국화재,보험업,손해보험,1974-12-05,12월,임규준,http://www.insurance.co.kr,서울특별시
7853,003280,KOSPI,흥아해운,해상 운송업,외항화물운송업(케미컬탱커),1976-06-29,12월,이환구,http://www.heung-a.com,서울특별시
7854,037440,KOSDAQ,희림,"건축기술, 엔지니어링 및 관련 기술 서비스업",설계 및 감리용역,2000-02-03,12월,"정영균, 이목운, 허철호 (각자대표)",http://www.heerim.com,서울특별시


In [3]:
import time
import pandas as pd

In [25]:
#수정주가를 데이터베이스에 삽입하는 함수입니다.
def inputAsp(dates):
    daily = pd.DataFrame()
    for code, name in stock_list[['Symbol', 'Name']].values:
        date = dates
        ohlcv = fdr.DataReader(code, date, date)
        ohlcv['Code'] = code
        ohlcv['Name'] = name
        daily= pd.concat([daily, ohlcv])
        daily['Date'] = date
        
    daily.to_sql(name='asp',con=engine,if_exists='append',index=False,
         dtype = { # sql에 저장할 때, 데이터 유형도 설정할 수 있습니다.

             'Open' : sqlalchemy.types.BIGINT(),
             'High' : sqlalchemy.types.BIGINT(),
             'Low' : sqlalchemy.types.BIGINT(),
             'Close' : sqlalchemy.types.BIGINT(),
             'Volume' : sqlalchemy.types.BIGINT(),
             'Change' : sqlalchemy.types.FLOAT(),
             'Code' : sqlalchemy.types.VARCHAR(10),
             'Name' : sqlalchemy.types.TEXT(),
             'Date' : sqlalchemy.types.VARCHAR(20),

         }
        )
    
    print(date)

In [26]:
server = 'localhost'
user = 'root'
password = 'root'
db = 'mhdb'

engine = create_engine('mysql+pymysql://{}:{}@{}/{}?charset=utf8'.format(user,password,server,db))

#수정주가의 시작 날짜와 마지막 날짜를 정합니다. (5년간의 데이터)
start = "2017-06-07"
last = "2022-06-06"

start_date = datetime.strptime(start, "%Y-%m-%d")
last_date = datetime.strptime(last, "%Y-%m-%d")

#시작일부터 마지막일까지 inputAsp함수를 이용해 데이터베이스에 저장합니다.
while start_date <= last_date:
    dates = start_date.strftime("%Y-%m-%d")
    start_date += timedelta(days=1)
    inputAsp(dates)

engine.dispose()
#밑의 결과는 2017-06-07부터 2018-05-31까지 실행한 후 중지를 해서 오류가 뜬것이고
#나머지 부분은 다른파일과 병행해 실행해서 데이터베이스에 5년치를 정상적으로 삽입했습니다.

2017-06-07
2017-06-08
2017-06-09
2017-06-10
2017-06-11
2017-06-12
2017-06-13
2017-06-14
2017-06-15
2017-06-16
2017-06-17
2017-06-18
2017-06-19
2017-06-20
2017-06-21
2017-06-22
2017-06-23
2017-06-24
2017-06-25
2017-06-26
2017-06-27
2017-06-28
2017-06-29
2017-06-30
2017-07-01
2017-07-02
2017-07-03
2017-07-04
2017-07-05
2017-07-06
2017-07-07
2017-07-08
2017-07-09
2017-07-10
2017-07-11
2017-07-12
2017-07-13
2017-07-14
2017-07-15
2017-07-16
2017-07-17
2017-07-18
2017-07-19
2017-07-20
2017-07-21
2017-07-22
2017-07-23
2017-07-24
2017-07-25
2017-07-26
2017-07-27
2017-07-28
2017-07-29
2017-07-30
2017-07-31
2017-08-01
2017-08-02
2017-08-03
2017-08-04
2017-08-05
2017-08-06
2017-08-07
2017-08-08
2017-08-09
2017-08-10
2017-08-11
2017-08-12
2017-08-13
2017-08-14
2017-08-15
2017-08-16
2017-08-17
2017-08-18
2017-08-19
2017-08-20
2017-08-21
2017-08-22
2017-08-23
2017-08-24
2017-08-25
2017-08-26
2017-08-27
2017-08-28
2017-08-29
2017-08-30
2017-08-31
2017-09-01
2017-09-02
2017-09-03
2017-09-04
2017-09-05

KeyboardInterrupt: 

In [4]:
import pandas as pd
import numpy as np
import time, json, requests 
from datetime import datetime, timedelta, date

import pymysql # python에서 mysql을 사용하는 패키지

import sqlalchemy # sql 접근 및 관리를 도와주는 패키지
from sqlalchemy import create_engine

In [6]:
#krx에서 얻어온 stock_list에서 code만을 추출해 symbol_code에 삽입합니다.
symbol_code = []
for i in range(len(stock_list)):
    symbol_code.append(stock_list['Symbol'].values[i])
symbol_code

['060310',
 '095570',
 '006840',
 '054620',
 '265520',
 '211270',
 '027410',
 '282330',
 '032790',
 '138930',
 '001460',
 '013720',
 '001040',
 '079160',
 '035760',
 '311690',
 '000120',
 '011150',
 '097950',
 '051500',
 '058820',
 '023460',
 '056730',
 '065770',
 '083660',
 '000590',
 '012030',
 '016610',
 '005830',
 '000990',
 '139130',
 '001530',
 '000210',
 '001880',
 '068790',
 '004840',
 '241520',
 '155660',
 '069730',
 '017940',
 '245620',
 '037370',
 '278990',
 '365550',
 '050120',
 '084180',
 '383220',
 '007700',
 '214270',
 '130500',
 '114090',
 '083450',
 '006360',
 '001250',
 '007070',
 '297890',
 '078150',
 '012630',
 '039570',
 '089470',
 '294870',
 '267250',
 '097230',
 '195940',
 '028300',
 '003580',
 '343090',
 '067630',
 '047920',
 '011200',
 '036640',
 '082740',
 '335870',
 '388790',
 '003560',
 '095340',
 '099520',
 '175330',
 '950170',
 '234080',
 '067290',
 '001060',
 '096760',
 '035900',
 '318000',
 '024840',
 '105560',
 '024120',
 '002380',
 '021320',
 '344820',

In [5]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import time

In [8]:
#저장된 code를 이용해 각 종목별로 fnguide에서 제무재표를 크롤링해옵니다.
if len(symbol_code) > 0:
    url = []
    for i in range(len(symbol_code)):
        url.append(("https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=D&NewMenuID=103&stkGb=701".format(symbol_code[i])))
else:
    pass

In [9]:
headers = {
    "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.151 Whale/3.14.134.62 Safari/537.36"
}
if len(url) > 0:
    req = []
    for i in range(len(url)):
        req.append(Request(url = url[i], headers = headers))
else:
    pass

In [10]:
if len(url) > 0:
    html = []
    for i in range(len(url)):
        html.append(urlopen(req[i]).read())
else:
    pass

In [11]:
if len(url) > 0:
    soup = []
    for i in range(len(url)):
        soup.append(BeautifulSoup(html[i], 'html.parser'))
else:
    pass

In [12]:
new_soup = []
for i in range(len(soup)):
    new_soup.append(soup[i])

In [13]:
is_a = []
num_col = 4
for i in range(len(new_soup)):
    is_a.append(new_soup[i].find(id = 'divSonikY')) #연간데이터 표

In [13]:
is_a = []
num_col = 4
for i in range(len(new_soup)):
    is_a.append(new_soup[i].find(id = 'divSonikQ')) #분기데이터 표

In [14]:
if len(is_a) > 0:
    for i in range(len(is_a)):
        is_a[i] = is_a[i].find_all(['tr'])
else:
    pass

In [15]:
items_kr = []
for i in range(len(is_a)):
    items_kr.append([is_a[i][m].find('th').get_text().replace('\n','').replace('\xa0','').replace('계산에 참여한 계정 펼치기','')\
            for m in range(1, len(is_a[i]))])

In [16]:
period1 = []
for i in range(len(is_a)):
    period1.append([is_a[i][0].find_all('th')[n].get_text() for n in range(1, num_col+1)])

In [17]:
items_en = ['rev', 'cgs','gross',
            'sga', 'sgal', 'sga2', 'sga3', 'sga4', 'sga5', 'sgab', 'sga7', 'sga8', 'opr', 'opr_',
            'fininc','fininc1','finince','fininc3','fininc4', 'fininc5',
            'fininc6','fininc7', 'fininc8', 'finince9','fininc10', 'fininc11',
            'fincost','fincost1','fincost2','fincost3','fincost4', 'fincost5',
            'fincost6', 'fincost7', 'fincost8', 'fincost9', 'fincost 10',
            'otherrev', 'otherrev1', 'otherrev2', 'otherrev3', 'otherrev4', 'otherrev5', 'otherrev6', 'otherrev7', 'otherrev8',
            'otherrev9', 'otherrev10', 'otherrev11', 'otherrev12', 'otherrev13', 'otherrev14', 'otherrev15', 'otherrev16',
            'othercost', 'othercost1', 'othercost2', 'othercost3', 'othercost4', 'othercost5',
            'othercost6', 'othercost7', 'othercost8', 'othercost9', 'othercost 10', 'othercost11', 'othercost12',
            'otherpl', 'otherpil', 'otherp12', 'otherpi3', 'otherp/4',
            'ebit', 'tax', 'contop', 'discontop', 'netinc']
items_en = items_en + ['netinc1', 'netinc2']

In [18]:
import pymysql

In [21]:
#연간 데이터
new_symbol_code = []
new_soup2 = []
for k in range(len(is_a)):
    
    for item, i in zip(items_en, range(1, len(is_a[k]))):
        temps = []
        for j in range(0, num_col):
            temp = [float(is_a[k][i].find_all('td')[j]['title'].replace(',','').replace('\xa0',''))\
                   if is_a[k][i].find_all('td')[j]['title'].replace(',','').replace('\xa0','') != ''\
                   else (0 if is_a[k][i].find_all('td')[j]['title'].replace(',','').replace('\xa0','') == '-0' \
                        else 0)]
            temps.append(temp[0])
        globals()[item] = temps
        
    is_domestic = pd.DataFrame({'종목':symbol_code[k], '기간':period1[k],
                       '매출액':rev, '매출원가':cgs, '매출총이익': gross,
                       '판매비와관리비': sga,
                       '영업이익': opr, '영업이익(발표기준)':opr_,
                       '금융이익': fininc,
                       '금융원가': fincost,
                       '기타수익': otherrev,
                       '기타비용': othercost,
                       '관계기업관련손익': otherpl,
                       'EBIT': ebit, '법인세비용': tax, '계속영업이익': contop,
                       '중단영업이익': discontop, '당기순이익': netinc,
                       '지배주주순이익': globals()['netinc1'],
                       '비지배주주순이익': globals()['netinc2']})

    conn = pymysql.connect(host='localhost', user='root', password='root', db='mhdb', charset='utf8')
    curs = conn.cursor()
    
    #가끔 다른 표를 가진 데이터들이 있어서 지배주주순이익에 모든 연도가 같은 값이 저장되는 경우가 있습니다.
    #그러한 데이터를 구분하는 코드입니다.
    flag = 0
    val = is_domestic.지배주주순이익.values[0]
    for l in range(len(is_domestic)):
        if is_domestic.지배주주순이익.values[l] == val:
            flag = 1
        else:
            flag = 0
            break
    if flag == 1:   #지배주주순이익이 모두 같은 데이터는 데이터베이스에 저장을 건너뜁니다.
        continue
    if flag == 0:   #정상적인 데이터의 경우는 재무상태표와 현금흐름표도 저장을 해줘야하므로 값을 따로 저장합니다.
        new_symbol_code.append(symbol_code[k])
        new_soup2.append(new_soup[k])

    for idx in range(len(is_domestic)):
        stock_code = is_domestic.종목.values[idx]
        period = is_domestic.기간.values[idx]
        rev = is_domestic.매출액.values[idx]
        cgs = is_domestic.매출원가.values[idx]
        gross = is_domestic.매출총이익.values[idx]
        sga = is_domestic.판매비와관리비.values[idx]
        opr = is_domestic.영업이익.values[idx]
        opr_ = is_domestic.영업이익.values[idx]
        fininc = is_domestic.금융이익.values[idx]
        fincost = is_domestic.금융원가.values[idx]
        otherrev = is_domestic.기타수익.values[idx]
        othercost = is_domestic.기타비용.values[idx]
        otherpl = is_domestic.관계기업관련손익.values[idx]
        ebit = is_domestic.EBIT.values[idx]
        tax = is_domestic.법인세비용.values[idx]
        contop = is_domestic.계속영업이익.values[idx]
        discontop = is_domestic.중단영업이익.values[idx]
        netinc = is_domestic.당기순이익.values[idx]
        netinc1 = is_domestic.지배주주순이익.values[idx]
        netinc2 = is_domestic.비지배주주순이익.values[idx]

        #손익계산서 테이블에 데이터를 삽입하는 쿼리입니다.
        sql = f"REPLACE INTO income_statement (stock_code, period,rev,cgs,gross,sga,opr,opr_,fininc,fincost,otherrev,othercost,otherpl,ebit,tax,contop,discontop,netinc,netinc1,netinc2) VALUES ('{stock_code}', '{period}', '{rev}','{cgs}','{gross}','{sga}','{opr}','{opr_}', '{fininc}','{fincost}','{otherrev}','{othercost}', '{otherpl}', '{ebit}','{tax}','{contop}','{discontop}','{netinc}','{netinc1}','{netinc2}')"

        curs.execute(sql)

    conn.commit()
    conn.close()

In [20]:
#분기 트레일링 데이터
new_symbol_code = []
new_soup2 = []
for k in range(len(is_a)):
    
    for item, i in zip(items_en, range(1, len(is_a[k]))):
        temps = []
        for j in range(0, num_col):
            temp = [float(is_a[k][i].find_all('td')[j]['title'].replace(',','').replace('\xa0',''))\
                   if is_a[k][i].find_all('td')[j]['title'].replace(',','').replace('\xa0','') != ''\
                   else (0 if is_a[k][i].find_all('td')[j]['title'].replace(',','').replace('\xa0','') == '-0' \
                        else 0)]
            temps.append(temp[0])
        globals()[item] = temps
        
    is_domestic = pd.DataFrame({'종목':symbol_code[k], '기간':period1[k],
                       '매출액':rev, '매출원가':cgs, '매출총이익': gross,
                       '판매비와관리비': sga,
                       '영업이익': opr, '영업이익(발표기준)':opr_,
                       '금융이익': fininc,
                       '금융원가': fincost,
                       '기타수익': otherrev,
                       '기타비용': othercost,
                       '관계기업관련손익': otherpl,
                       'EBIT': ebit, '법인세비용': tax, '계속영업이익': contop,
                       '중단영업이익': discontop, '당기순이익': netinc,
                       '지배주주순이익': globals()['netinc1'],
                       '비지배주주순이익': globals()['netinc2']})

    conn = pymysql.connect(host='localhost', user='root', password='root', db='mhdb', charset='utf8')
    curs = conn.cursor()
    
    flag = 0
    val = is_domestic.지배주주순이익.values[0]
    for l in range(len(is_domestic)):
        if is_domestic.지배주주순이익.values[l] == val:
            flag = 1
        else:
            flag = 0
            break
    if flag == 1:
        continue
    if flag == 0:
        new_symbol_code.append(symbol_code[k])
        new_soup2.append(new_soup[k])

    #트레일링된 데이터를 구하기위해, 분기 데이터들을 돌면서 더해줄 변수를 초기화 해줍니다.
    revT = 0
    cgsT = 0
    grossT = 0
    sgaT = 0
    oprT = 0
    opr_T = 0
    finincT = 0
    fincostT = 0
    otherrevT = 0
    othercostT = 0
    otherplT = 0
    ebitT = 0
    taxT = 0
    contopT = 0
    discontopT = 0
    netincT = 0
    netinc1T = 0
    netinc2T = 0
    for idx in range(len(is_domestic)):
        #분기 데이터들을 돌면서 값을 더해줍니다.
        stock_code = is_domestic.종목.values[idx]
        period = is_domestic.기간.values[idx]        
        revT += is_domestic.매출액.values[idx]
        cgsT += is_domestic.매출원가.values[idx]
        grossT += is_domestic.매출총이익.values[idx]
        sgaT += is_domestic.판매비와관리비.values[idx]
        oprT += is_domestic.영업이익.values[idx]
        opr_T += is_domestic.영업이익.values[idx]
        finincT += is_domestic.금융이익.values[idx]
        fincostT += is_domestic.금융원가.values[idx]
        otherrevT += is_domestic.기타수익.values[idx]
        othercostT += is_domestic.기타비용.values[idx]
        otherplT += is_domestic.관계기업관련손익.values[idx]
        ebitT += is_domestic.EBIT.values[idx]
        taxT += is_domestic.법인세비용.values[idx]
        contopT += is_domestic.계속영업이익.values[idx]
        discontopT += is_domestic.중단영업이익.values[idx]
        netincT += is_domestic.당기순이익.values[idx]
        netinc1T += is_domestic.지배주주순이익.values[idx]
        netinc2T += is_domestic.비지배주주순이익.values[idx]

        #마지막 분기에 트레일링된 데이터를 테이블에 삽입합니다.
        if idx == (len(is_domestic) - 1):
            sql = f"REPLACE INTO income_statement_quarter (stock_code, period,rev,cgs,gross,sga,opr,opr_,fininc,fincost,otherrev,othercost,otherpl,ebit,tax,contop,discontop,netinc,netinc1,netinc2) VALUES ('{stock_code}', '{period}', '{revT}','{cgsT}','{grossT}','{sgaT}','{oprT}','{opr_T}', '{finincT}','{fincostT}','{otherrevT}','{othercostT}', '{otherplT}', '{ebitT}','{taxT}','{contopT}','{discontopT}','{netincT}','{netinc1T}','{netinc2T}')"
        
            curs.execute(sql)

    conn.commit()
    conn.close()

In [25]:
bs_a = []
num_col = 4
for i in range(len(new_soup2)):
    bs_a.append(new_soup2[i].find(id = 'divDaechaY')) #연간 데이터 표

In [21]:
bs_a = []
num_col = 4
for i in range(len(new_soup2)):
    bs_a.append(new_soup2[i].find(id = 'divDaechaQ')) #분기 데이터 표

In [22]:
if len(bs_a) > 0:
    for i in range(len(bs_a)):
        bs_a[i] = bs_a[i].find_all(['tr'])
else:
    pass

In [23]:
items_kr1 = []
for i in range(len(bs_a)):
    items_kr1.append([bs_a[i][m].find('th').get_text().replace('\n','').replace('\xa0','').replace('계산에 참여한 계정 펼치기','')\
            for m in range(1, len(bs_a[i]))])

In [24]:
period2 = []
for i in range(len(bs_a)):
    period2.append([bs_a[i][0].find_all('th')[n].get_text() for n in range(1, num_col+1)])

In [31]:
items_en = ['assets', 'curassets', 'curassets1', 'curassets2', 'curassets3', 
            'curassets4', 'curassets5', 'curassets6', 'curassets7',
            'curassets8', 'curassets9', 'curassets10', 'curassets11',
            'Itassets', 'Itassets1', 'Itassets2', 'Itassets3' , 'Itassets4',
            'Itassets5', 'Itassets6', 'Itassets7' , 'Itassets8', 'Itassets9',
            'Itassets10', 'Itassets11', 'Itassets12', 'Itassets13' , 
            'finassets', 'liab', 'curliab', 'curliab1', 'curliab2', 'curliab3',
            'curliab4', 'curliab5', 'curliab6', 'curliab7', 'curliab8', 
            'curliab9', 'curliab10', 'curliab11', 'curliab12', 'curliab13',
            'ltliab', 'ltliab1','ltliab2', 'ltliab3', 'ltliab4', 'ltliab5',
            'ltliab6', 'ltliab7', 'ltliab8', 'ltliab9', 'ltliab10', 'ltliab11',
            'ltliab12', 'finliab', 'equity', 'equity1', 'equity2', 'equity3',
            'equity4', 'equity5', 'equity6', 'equity7', 'equity8']

In [31]:
#연간 데이터
#재무상태표의 연간데이터를 데이터베이스에 저장합니다.
for k in range(len(bs_a)):
    
    for item, i in zip(items_en, range(1, len(bs_a[k]))):
        temps = []
        for j in range(0, num_col):
            temp = [float(bs_a[k][i].find_all('td')[j]['title'].replace(',','').replace('\xa0',''))\
                   if bs_a[k][i].find_all('td')[j]['title'].replace(',','').replace('\xa0','') != ''\
                   else (0 if bs_a[k][i].find_all('td')[j]['title'].replace(',','').replace('\xa0','') == '-0' \
                        else 0)]
            temps.append(temp[0])
        globals()[item] = temps
        
    bs_domestic = pd. DataFrame({"stock_code" :new_symbol_code[k], "period": period2[k],
                            "Assets_Total": assets, "Current_Assets_Total": curassets,
                             "Current_Inventory":curassets1, "Current_Bio_Assets":curassets2, 
                             "Current_Fin_Assets" : curassets3, "Current_Receivables": curassets4, 
                             "Current_Tax_Assets" :curassets5, "Current_Entrusted_Assets":curassets6, 
                             "Current_Returned_Goods_Assets" : curassets7,"Current_Emissions_Allowance_Assets" : curassets8, 
                             "Other_Current_Assets":curassets9, "Cash_and_Cash_Equivalents":curassets10, 
                             "Current_Assets_On_Assets_For_Sale":curassets11,
                             "LT_Assets_Total":Itassets, "LT_Tangible_Assets":Itassets1, 
                             "LT_Intangible_Assets":Itassets2, "LT_Bio_Assets":Itassets3, 
                             "LT_InveCurrent_Real_Estate":Itassets4, "LT_Fin_Assets":Itassets5, 
                             "LT_InveCurrent_Associates": Itassets6, "LT_Receivables": Itassets7, 
                             "LT_Def_Income_Tax_Assets":Itassets8, "LT_Tax_Assets":Itassets9, 
                             "LT_Entrusted_Assets":Itassets10, "LT_Returned_Goods" : Itassets11, 
                             "LT_Emissions_Allowance" : Itassets12, "Other_LT_Assets":Itassets13, 
                             "Other_Fin_Assets":finassets,
                             "Liabilities_Total":liab, "Current_Liab_Total": curliab, "Current_Bonds" : curliab1, 
                             "Current_Loans": curliab2, "Current_Portion_LT_Liab": curliab3, 
                             "Current_Fin_Liab":curliab4, "Current_Payables":curliab5, 
                             "Current_Emp_Benefits": curliab6, "Current_Provisions":curliab7, 
                             "Current_Tax_Liab" :curliab8, "Current_Entrusted_Liab": curliab9, 
                             "Current_Returned_Goods_Liab":curliab10, "Current_Emissions_Allowance_Liab": curliab11, 
                             "Other_Current_Liab": curliab12, "Current_Liab_On_Assets_For_Sale" : curliab13, # 15 
                             "LT_Liab_Total":ltliab, "LT_Bonds":ltliab1, "LT_Loans": ltliab2, "LT_Fin_Liab":ltliab3, 
                             "LT_Payables":ltliab4, "LT_Emp_Benefits":ltliab5, "LT_Provisions":ltliab6, 
                             "LT_Def_Income_Tax_Liab":ltliab7, "LT_Tax_Liab":ltliab8, 
                             "LT_Entrusted_Liab":ltliab9, "LT_Returned_Goods_Liab":ltliab10, 
                             "LT_Emissions_Allowance_Liab":ltliab11, "Other_LT_Liab":ltliab12, 
                             "Other_Fin_Liab_Total":finliab,
                             "Equity_Total": equity, "Controlling_Equity_Total": equity1, "Paid_in_Capital": equity2, 
                             "Contingent_Convertible_Bonds": equity3, "Capital_Surplus" : equity4, 
                             "Other_Equity": equity5, "Accum_Other_Comprehensive_Income" :equity6, 
                             "Retained_Earnings": equity7, "Non_Controlling_Equity_total": equity8})

    conn = pymysql.connect(host='localhost', user='root', password='root', db='mhdb', charset='utf8')
    curs = conn.cursor()

    for idx in range(len(bs_domestic)):
        stock_code = bs_domestic.stock_code.values[idx]
        period = bs_domestic.period.values[idx]
        assets = bs_domestic.Assets_Total.values[idx]
        curassets = bs_domestic.Current_Assets_Total.values[idx]
        curassets1 = bs_domestic.Current_Inventory.values[idx]
        curassets2 = bs_domestic.Current_Bio_Assets.values[idx]
        curassets3 = bs_domestic.Current_Fin_Assets.values[idx]
        curassets4 = bs_domestic.Current_Receivables.values[idx]
        curassets5 = bs_domestic.Current_Tax_Assets.values[idx]
        curassets6 = bs_domestic.Current_Entrusted_Assets.values[idx]
        curassets7 = bs_domestic.Current_Returned_Goods_Assets.values[idx]
        curassets8 = bs_domestic.Current_Emissions_Allowance_Assets.values[idx]
        curassets9 = bs_domestic.Other_Current_Assets.values[idx]
        curassets10 = bs_domestic.Cash_and_Cash_Equivalents.values[idx]
        curassets11 = bs_domestic.Current_Assets_On_Assets_For_Sale.values[idx]
        
        Itassets = bs_domestic.LT_Assets_Total.values[idx]
        Itassets1 = bs_domestic.LT_Tangible_Assets.values[idx]
        Itassets2 = bs_domestic.LT_Intangible_Assets.values[idx]
        Itassets3 = bs_domestic.LT_Bio_Assets.values[idx]
        Itassets4 = bs_domestic.LT_InveCurrent_Real_Estate.values[idx]
        Itassets5 = bs_domestic.LT_Fin_Assets.values[idx]
        Itassets6 = bs_domestic.LT_InveCurrent_Associates.values[idx]
        Itassets7 = bs_domestic.LT_Receivables.values[idx]
        Itassets8 = bs_domestic.LT_Def_Income_Tax_Assets.values[idx]
        Itassets9 = bs_domestic.LT_Tax_Assets.values[idx]
        Itassets10 = bs_domestic.LT_Entrusted_Assets.values[idx]
        Itassets11 = bs_domestic.LT_Returned_Goods.values[idx]
        Itassets12 = bs_domestic.LT_Emissions_Allowance.values[idx]
        Itassets13 = bs_domestic.Other_LT_Assets.values[idx]
        
        finassets = bs_domestic.Other_Fin_Assets.values[idx]
        
        liab = bs_domestic.Liabilities_Total.values[idx]
        curliab = bs_domestic.Current_Liab_Total.values[idx]
        curliab1 = bs_domestic.Current_Bonds.values[idx]
        curliab2 = bs_domestic.Current_Loans.values[idx]
        curliab3 = bs_domestic.Current_Portion_LT_Liab.values[idx]
        curliab4 = bs_domestic.Current_Fin_Liab.values[idx]
        curliab5 = bs_domestic.Current_Payables.values[idx]
        curliab6 = bs_domestic.Current_Emp_Benefits.values[idx]
        curliab7 = bs_domestic.Current_Provisions.values[idx]
        curliab8 = bs_domestic.Current_Tax_Liab.values[idx]
        curliab9 = bs_domestic.Current_Entrusted_Liab.values[idx]
        curliab10 = bs_domestic.Current_Returned_Goods_Liab.values[idx]
        curliab11 = bs_domestic.Current_Emissions_Allowance_Liab.values[idx]
        curliab12 = bs_domestic.Other_Current_Liab.values[idx]
        curliab13 = bs_domestic.Current_Liab_On_Assets_For_Sale.values[idx]
        
        ltliab = bs_domestic.LT_Liab_Total.values[idx]
        ltliab1 = bs_domestic.LT_Bonds.values[idx]
        ltliab2 = bs_domestic.LT_Loans.values[idx]
        ltliab3 = bs_domestic.LT_Fin_Liab.values[idx]
        ltliab4 = bs_domestic.LT_Payables.values[idx]
        ltliab5 = bs_domestic.LT_Emp_Benefits.values[idx]
        ltliab6 = bs_domestic.LT_Provisions.values[idx]
        ltliab7 = bs_domestic.LT_Def_Income_Tax_Liab.values[idx]
        ltliab8 = bs_domestic.LT_Tax_Liab.values[idx]
        ltliab9 = bs_domestic.LT_Entrusted_Liab.values[idx]
        ltliab10 = bs_domestic.LT_Returned_Goods_Liab.values[idx]
        ltliab11 = bs_domestic.LT_Emissions_Allowance_Liab.values[idx]
        ltliab12 = bs_domestic.Other_LT_Liab.values[idx]
        
        finliab = bs_domestic.Other_Fin_Liab_Total.values[idx]
        
        equity = bs_domestic.Equity_Total.values[idx]
        equity1 = bs_domestic.Controlling_Equity_Total.values[idx]
        equity2 = bs_domestic.Paid_in_Capital.values[idx]
        equity3 = bs_domestic.Contingent_Convertible_Bonds.values[idx]
        equity4 = bs_domestic.Capital_Surplus.values[idx]
        equity5 = bs_domestic.Other_Equity.values[idx]
        equity6 = bs_domestic.Accum_Other_Comprehensive_Income.values[idx]
        equity7 = bs_domestic.Retained_Earnings.values[idx]
        equity8 = bs_domestic.Non_Controlling_Equity_total.values[idx]

        sql = f"REPLACE INTO balance_statement VALUES ('{stock_code}', '{period}', '{assets}','{curassets}','{curassets1}','{curassets2}','{curassets3}','{curassets4}','{curassets5}','{curassets6}','{curassets7}','{curassets8}', '{curassets9}', '{curassets10}','{curassets11}','{Itassets}','{Itassets1}','{Itassets2}','{Itassets3}','{Itassets4}','{Itassets5}', '{Itassets6}', '{Itassets7}', '{Itassets8}', '{Itassets9}', '{Itassets10}', '{Itassets11}', '{Itassets12}', '{Itassets13}','{finassets}','{liab}','{curliab}','{curliab1}','{curliab2}','{curliab3}','{curliab4}','{curliab5}','{curliab6}','{curliab7}','{curliab8}','{curliab9}','{curliab10}','{curliab11}','{curliab12}','{curliab13}','{ltliab}','{ltliab1}','{ltliab2}','{ltliab3}','{ltliab4}','{ltliab5}','{ltliab6}','{ltliab7}','{ltliab8}','{ltliab9}','{ltliab10}','{ltliab11}','{ltliab12}','{finliab}','{equity}','{equity1}','{equity2}','{equity3}','{equity4}','{equity5}','{equity6}','{equity7}','{equity8}')"

        curs.execute(sql)

    conn.commit()
    conn.close()

In [33]:
#분기 트레일링 데이터
#재무상태표의 분기 데이터의 트레일링된 데이터를 데이터베이스에 저장합니다.
for k in range(len(bs_a)):
    
    for item, i in zip(items_en, range(1, len(bs_a[k]))):
        temps = []
        for j in range(0, num_col):
            temp = [float(bs_a[k][i].find_all('td')[j]['title'].replace(',','').replace('\xa0',''))\
                   if bs_a[k][i].find_all('td')[j]['title'].replace(',','').replace('\xa0','') != ''\
                   else (0 if bs_a[k][i].find_all('td')[j]['title'].replace(',','').replace('\xa0','') == '-0' \
                        else 0)]
            temps.append(temp[0])
        globals()[item] = temps
        
    bs_domestic = pd. DataFrame({"stock_code" :new_symbol_code[k], "period": period2[k],
                            "Assets_Total": assets, "Current_Assets_Total": curassets,
                             "Current_Inventory":curassets1, "Current_Bio_Assets":curassets2, 
                             "Current_Fin_Assets" : curassets3, "Current_Receivables": curassets4, 
                             "Current_Tax_Assets" :curassets5, "Current_Entrusted_Assets":curassets6, 
                             "Current_Returned_Goods_Assets" : curassets7,"Current_Emissions_Allowance_Assets" : curassets8, 
                             "Other_Current_Assets":curassets9, "Cash_and_Cash_Equivalents":curassets10, 
                             "Current_Assets_On_Assets_For_Sale":curassets11,
                             "LT_Assets_Total":Itassets, "LT_Tangible_Assets":Itassets1, 
                             "LT_Intangible_Assets":Itassets2, "LT_Bio_Assets":Itassets3, 
                             "LT_InveCurrent_Real_Estate":Itassets4, "LT_Fin_Assets":Itassets5, 
                             "LT_InveCurrent_Associates": Itassets6, "LT_Receivables": Itassets7, 
                             "LT_Def_Income_Tax_Assets":Itassets8, "LT_Tax_Assets":Itassets9, 
                             "LT_Entrusted_Assets":Itassets10, "LT_Returned_Goods" : Itassets11, 
                             "LT_Emissions_Allowance" : Itassets12, "Other_LT_Assets":Itassets13, 
                             "Other_Fin_Assets":finassets,
                             "Liabilities_Total":liab, "Current_Liab_Total": curliab, "Current_Bonds" : curliab1, 
                             "Current_Loans": curliab2, "Current_Portion_LT_Liab": curliab3, 
                             "Current_Fin_Liab":curliab4, "Current_Payables":curliab5, 
                             "Current_Emp_Benefits": curliab6, "Current_Provisions":curliab7, 
                             "Current_Tax_Liab" :curliab8, "Current_Entrusted_Liab": curliab9, 
                             "Current_Returned_Goods_Liab":curliab10, "Current_Emissions_Allowance_Liab": curliab11, 
                             "Other_Current_Liab": curliab12, "Current_Liab_On_Assets_For_Sale" : curliab13, # 15 
                             "LT_Liab_Total":ltliab, "LT_Bonds":ltliab1, "LT_Loans": ltliab2, "LT_Fin_Liab":ltliab3, 
                             "LT_Payables":ltliab4, "LT_Emp_Benefits":ltliab5, "LT_Provisions":ltliab6, 
                             "LT_Def_Income_Tax_Liab":ltliab7, "LT_Tax_Liab":ltliab8, 
                             "LT_Entrusted_Liab":ltliab9, "LT_Returned_Goods_Liab":ltliab10, 
                             "LT_Emissions_Allowance_Liab":ltliab11, "Other_LT_Liab":ltliab12, 
                             "Other_Fin_Liab_Total":finliab,
                             "Equity_Total": equity, "Controlling_Equity_Total": equity1, "Paid_in_Capital": equity2, 
                             "Contingent_Convertible_Bonds": equity3, "Capital_Surplus" : equity4, 
                             "Other_Equity": equity5, "Accum_Other_Comprehensive_Income" :equity6, 
                             "Retained_Earnings": equity7, "Non_Controlling_Equity_total": equity8})

    conn = pymysql.connect(host='localhost', user='root', password='root', db='mhdb', charset='utf8')
    curs = conn.cursor()
    
    equity1T = 0

    for idx in range(len(bs_domestic)):
        stock_code = bs_domestic.stock_code.values[idx]
        period = bs_domestic.period.values[idx]
        equity1T += bs_domestic.Controlling_Equity_Total.values[idx]

        if idx == (len(bs_domestic) - 1):
            sql = f"REPLACE INTO balance_statement_trailing (stock_code, period, Controlling_Equity_Total) VALUES ('{stock_code}', '{period}', '{equity1T}')"

            curs.execute(sql)

    conn.commit()
    conn.close()

In [32]:
cf_a = []
num_col = 4
for i in range(len(new_soup2)):
    cf_a.append(new_soup2[i].find(id = 'divCashY')) #연간 데이터 표

In [ ]:
cf_a = []
num_col = 4
for i in range(len(new_soup2)):
    cf_a.append(new_soup2[i].find(id = 'divCashQ')) #분기 데이터 표

In [33]:
if len(cf_a) > 0:
    for i in range(len(cf_a)):
        cf_a[i] = cf_a[i].find_all(['tr'])
else:
    pass

In [34]:
items_kr2 = []
for i in range(len(cf_a)):
    items_kr2.append([cf_a[i][m].find('th').get_text().replace('\n','').replace('\xa0','').replace('계산에 참여한 계정 펼치기','')\
            for m in range(1, len(cf_a[i]))])

In [35]:
period3 = []
for i in range(len(cf_a)):
    period3.append([cf_a[i][0].find_all('th')[n].get_text() for n in range(1, num_col+1)])

In [36]:
items_en = ["cfo","cfo1","cfo2","cfo3","cfo4","cfo5","cfo6","cfo7"
           ,"cfi","cfi1","cfi2","cfi3","cff","cff1","cff2","cff3"
           ,"cff4","cff5","cff6","cff7","cff8","cff9"]

In [37]:
#연간 데이터
#현금흐름표의 연간데이터를 데이터베이스에 저장합니다.
for k in range(len(cf_a)):
    
    for item, i in zip(items_en, range(1, len(cf_a[k]))):
        temps = []
        for j in range(0, num_col):
            temp = [float(cf_a[k][i].find_all('td')[j]['title'].replace(',','').replace('\xa0',''))\
                   if cf_a[k][i].find_all('td')[j]['title'].replace(',','').replace('\xa0','') != ''\
                   else (0 if cf_a[k][i].find_all('td')[j]['title'].replace(',','').replace('\xa0','') == '-0' \
                        else 0)]
            temps.append(temp[0])
        globals()[item] = temps
        
    cf_domestic = pd.DataFrame({"stock_code": new_symbol_code[k], "period": period3[k],
                           "CFO_Total": cfo, "Net_Income_Total":cfo1, "Cont_Biz_Before_Tax":cfo2,
                           "Add_Exp_WO_CF_Out":cfo3, "Ded_Rev_WO_CF_In":cfo4, "Chg_Working_Capital":cfo5,
                           "CFO":cfo6, "Other_CFO":cfo7,
                           "CFI_Total":cfi, "CFI_In":cfi1, "CFI_Out":cfi2, "Other_CFI":cfi3,
                           "CFF_Total":cff, "CFF_In":cff1, "CFF_Out":cff2, "Other_CFF":cff3,
                           "Other_CF":cff4, "Chg_CF_Consolidation":cff5, "Forex_Effect":cff6,
                           "Chg_Cash_and_Cash_Equivalents":cff7, "Cash_and_Cash_Equivalents_Beg":cff8,
                           "Cash_and_Cash_Equivalents_End":cff9})

    conn = pymysql.connect(host='localhost', user='root', password='root', db='mhdb', charset='utf8')
    curs = conn.cursor()

    for idx in range(len(cf_domestic)):
        stock_code = cf_domestic.stock_code.values[idx]
        period = cf_domestic.period.values[idx]
        
        cfo = cf_domestic.CFO_Total.values[idx]
        cfo1 = cf_domestic.Net_Income_Total.values[idx]
        cfo2 = cf_domestic.Cont_Biz_Before_Tax.values[idx]
        cfo3 = cf_domestic.Add_Exp_WO_CF_Out.values[idx]
        cfo4 = cf_domestic.Ded_Rev_WO_CF_In.values[idx]
        cfo5 = cf_domestic.Chg_Working_Capital.values[idx]
        cfo6 = cf_domestic.CFO.values[idx]
        cfo7 = cf_domestic.Other_CFO.values[idx]
        
        cfi = cf_domestic.CFI_Total.values[idx]
        cfi1 = cf_domestic.CFI_In.values[idx]
        cfi2 = cf_domestic.CFI_Out.values[idx]
        cfi3 = cf_domestic.Other_CFI.values[idx]
        
        cff = cf_domestic.CFF_Total.values[idx]
        cff1 = cf_domestic.CFF_In.values[idx]
        cff2 = cf_domestic.CFF_Out.values[idx]
        cff3 = cf_domestic.Other_CFF.values[idx]
        cff4 = cf_domestic.Other_CF.values[idx]
        cff5 = cf_domestic.Chg_CF_Consolidation.values[idx]
        cff6 = cf_domestic.Forex_Effect.values[idx]
        cff7 = cf_domestic.Chg_Cash_and_Cash_Equivalents.values[idx]
        cff8 = cf_domestic.Cash_and_Cash_Equivalents_Beg.values[idx]
        cff9 = cf_domestic.Cash_and_Cash_Equivalents_End.values[idx]

        sql = f"REPLACE INTO cash_statement VALUES ('{stock_code}', '{period}', '{cfo}','{cfo1}','{cfo2}','{cfo3}','{cfo4}','{cfo5}', '{cfo6}','{cfo7}','{cfi}','{cfi1}', '{cfi2}', '{cfi3}','{cff}','{cff1}','{cff2}','{cff3}','{cff4}','{cff5}','{cff6}','{cff7}','{cff8}','{cff9}')"

        curs.execute(sql)

    conn.commit()
    conn.close()

In [8]:
#코드별로 주식수를 저장하기 위해 fnguide의 snapshot파트를 크롤링합니다.
if len(symbol_code) > 0:
    url = []
    for i in range(len(symbol_code)):
        url.append(("http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701".format(symbol_code[i])))
else:
    pass

In [9]:
headers = {
    "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.151 Whale/3.14.134.62 Safari/537.36"
}
if len(url) > 0:
    req = []
    for i in range(len(url)):
        req.append(Request(url = url[i], headers = headers))
else:
    pass

In [10]:
if len(url) > 0:
    html = []
    for i in range(len(url)):
        html.append(urlopen(req[i]).read())
else:
    pass

In [11]:
if len(url) > 0:
    soup = []
    for i in range(len(url)):
        soup.append(BeautifulSoup(html[i], 'html.parser'))
else:
    pass

In [12]:
new_soup = []
for i in range(len(soup)):
    new_soup.append(soup[i])

In [13]:
snap_a = []
num_col = 4
for i in range(len(new_soup)):
    snap_a.append(new_soup[i].find(id = 'svdMainGrid1')) #주식수가 포함된 표(시세현황)

In [14]:
if len(snap_a) > 0:
    for i in range(len(snap_a)):
        snap_a[i] = snap_a[i].find_all(['tr'])
else:
    pass

In [15]:
#시세현황 표의 주식수 중 보통주와 우선주를 더해 number배열에 저장합니다.
number = []
for i in range(len(snap_a)):
    temps = snap_a[i][6].find(['td']).getText()
    temp = temps.split('/ ')
    a = temp[0].split(',')
    b = temp[1].split(',')
    ta = ""
    tb = ""
    for j in range(len(a)):
        ta += a[j]
    for j in range(len(b)):
        tb += b[j]
    num = int(ta) + int(tb)
    number.append(num)

In [16]:
#얻어낸 주식수를 stock_number테이블에 저장합니다.
for i in range(len(number)):
    stock_code = symbol_code[i]
    stock_number = number[i]
    
    conn = pymysql.connect(host='localhost', user='root', password='root', db='mhdb', charset='utf8')
    curs = conn.cursor()
    
    sql = f"INSERT INTO stock_number VALUES ('{stock_code}', {stock_number})"
    curs.execute(sql)
    
    conn.commit()
    conn.close()

In [25]:
conn = pymysql.connect(host='localhost', user='root', password='root', db='mhdb', charset='utf8')
curs = conn.cursor()
sql = f"SELECT DISTINCT stock_code FROM balance_statement_quarter"
curs.execute(sql)
stock_code = curs.fetchall()

conn.close()

symbol_code = []
for i in range(len(stock_code)):
    symbol_code.append(stock_code[i][0])

In [26]:
print(len(symbol_code))

1783


In [40]:
conn = pymysql.connect(host='localhost', user='root', password='root', db='mhdb', charset='utf8')
curs = conn.cursor()
sql = f"SELECT * FROM balance_statement_quarter WHERE stock_code = '000020'"
curs.execute(sql)
balance_statement_quarter = curs.fetchall()

balance_statement_trailing = []
for i in range(len(balance_statement_quarter[len(balance_statement_quarter) - 1])):
    if i == 0 or i == 1:
        continue
    balance_statement_trailing.append(balance_statement_quarter[len(balance_statement_quarter) - 1][i])

for i in range(len(balance_statement_quarter) - 1):
    for j in range(len(balance_statement_quarter[i])):
        if j == 0 or j == 1:
            continue
        balance_statement_trailing[j - 2] += balance_statement_quarter[i][j]

conn.close()

In [44]:
print(balance_statement_quarter)
print(balance_statement_trailing)

(('000020', '2021/06', 4470.8, 2273.63, 431.09, 0.0, 790.0, 645.78, 1.0, 0.0, 3.23, 0.0, 13.22, 389.31, 0.0, 2197.16, 1290.62, 187.18, 0.0, 93.56, 473.14, 60.99, 0.0, 42.41, 0.0, 0.0, 0.0, 0.0, 49.27, 0.0, 959.99, 664.07, 0.0, 45.0, 0.0, 29.51, 477.25, 0.0, 0.0, 45.11, 0.0, 59.59, 0.0, 7.61, 0.0, 295.91, 0.0, 34.51, 35.49, 26.07, 172.87, 0.0, 26.49, 0.0, 0.0, 0.0, 0.0, 0.49, 0.0, 3510.81, 3388.13, 279.31, 0.0, 270.69, -25.21, 73.22, 2790.12, 122.68), ('000020', '2021/09', 4478.73, 2179.65, 419.64, 0.0, 706.1, 634.18, 0.34, 0.0, 3.2, 0.0, 8.28, 407.92, 0.0, 2299.08, 1390.15, 186.55, 0.0, 93.35, 458.3, 61.2, 0.0, 63.67, 0.0, 0.0, 0.0, 0.0, 45.87, 0.0, 947.32, 641.18, 0.0, 45.0, 0.0, 30.13, 447.35, 0.0, 0.0, 39.69, 0.0, 59.29, 0.0, 19.73, 0.0, 306.14, 0.0, 34.55, 31.11, 24.54, 188.59, 0.0, 26.9, 0.0, 0.0, 0.0, 0.0, 0.45, 0.0, 3531.4, 3404.95, 279.31, 0.0, 275.45, -19.65, 48.02, 2821.82, 126.45), ('000020', '2021/12', 4478.04, 2201.99, 362.44, 0.0, 610.3, 610.93, 0.01, 0.0, 3.64, 0.0, 9.87

In [45]:
for i in range(len(symbol_code)):
    conn = pymysql.connect(host='localhost', user='root', password='root', db='mhdb', charset='utf8')
    curs = conn.cursor()
    stock_code = symbol_code[i]
    sql = f"SELECT * FROM balance_statement_quarter WHERE stock_code = '{stock_code}'"
    curs.execute(sql)
    balance_statement_quarter = curs.fetchall()
    stock_code = balance_statement_quarter[len(balance_statement_quarter) - 1][0]
    period = balance_statement_quarter[len(balance_statement_quarter) - 1][1]
    conn.close()

    balance_statement_trailing = []
    for j in range(len(balance_statement_quarter[len(balance_statement_quarter) - 1])):
        if j == 0 or j == 1:
            continue
        balance_statement_trailing.append(balance_statement_quarter[len(balance_statement_quarter) - 1][j])

    for j in range(len(balance_statement_quarter) - 1):
        for k in range(len(balance_statement_quarter[j])):
            if k == 0 or k == 1:
                continue
            balance_statement_trailing[k - 2] += balance_statement_quarter[j][k]
    
    conn = pymysql.connect(host='localhost', user='root', password='root', db='mhdb', charset='utf8')
    curs = conn.cursor()
    
    sql = f"REPLACE INTO balance_statement_trailing VALUES ('{stock_code}', '{period}', '{balance_statement_trailing[0]}','{balance_statement_trailing[1]}','{balance_statement_trailing[2]}','{balance_statement_trailing[3]}','{balance_statement_trailing[4]}','{balance_statement_trailing[5]}','{balance_statement_trailing[6]}','{balance_statement_trailing[7]}','{balance_statement_trailing[8]}','{balance_statement_trailing[9]}', '{balance_statement_trailing[10]}', '{balance_statement_trailing[11]}','{balance_statement_trailing[12]}','{balance_statement_trailing[13]}','{balance_statement_trailing[14]}','{balance_statement_trailing[15]}','{balance_statement_trailing[16]}','{balance_statement_trailing[17]}','{balance_statement_trailing[18]}', '{balance_statement_trailing[19]}', '{balance_statement_trailing[20]}', '{balance_statement_trailing[21]}', '{balance_statement_trailing[22]}', '{balance_statement_trailing[23]}', '{balance_statement_trailing[24]}', '{balance_statement_trailing[25]}', '{balance_statement_trailing[26]}','{balance_statement_trailing[27]}','{balance_statement_trailing[28]}','{balance_statement_trailing[29]}','{balance_statement_trailing[30]}','{balance_statement_trailing[31]}','{balance_statement_trailing[32]}','{balance_statement_trailing[33]}','{balance_statement_trailing[34]}','{balance_statement_trailing[35]}','{balance_statement_trailing[36]}','{balance_statement_trailing[37]}','{balance_statement_trailing[38]}','{balance_statement_trailing[39]}','{balance_statement_trailing[40]}','{balance_statement_trailing[41]}','{balance_statement_trailing[42]}','{balance_statement_trailing[43]}','{balance_statement_trailing[44]}','{balance_statement_trailing[45]}','{balance_statement_trailing[46]}','{balance_statement_trailing[47]}','{balance_statement_trailing[48]}','{balance_statement_trailing[49]}','{balance_statement_trailing[50]}','{balance_statement_trailing[51]}','{balance_statement_trailing[52]}','{balance_statement_trailing[53]}','{balance_statement_trailing[54]}','{balance_statement_trailing[55]}','{balance_statement_trailing[56]}','{balance_statement_trailing[57]}','{balance_statement_trailing[58]}','{balance_statement_trailing[59]}','{balance_statement_trailing[60]}','{balance_statement_trailing[61]}','{balance_statement_trailing[62]}','{balance_statement_trailing[63]}','{balance_statement_trailing[64]}','{balance_statement_trailing[65]}')"
    curs.execute(sql)

    conn.commit()
    conn.close()

In [46]:
conn = pymysql.connect(host='localhost', user='root', password='root', db='mhdb', charset='utf8')
curs = conn.cursor()
sql = f"SELECT DISTINCT stock_code FROM cash_statement_quarter"
curs.execute(sql)
stock_code = curs.fetchall()

conn.close()

symbol_code = []
for i in range(len(stock_code)):
    symbol_code.append(stock_code[i][0])

In [47]:
print(len(symbol_code))

1783


In [49]:
for i in range(len(symbol_code)):
    conn = pymysql.connect(host='localhost', user='root', password='root', db='mhdb', charset='utf8')
    curs = conn.cursor()
    stock_code = symbol_code[i]
    sql = f"SELECT * FROM cash_statement_quarter WHERE stock_code = '{stock_code}'"
    curs.execute(sql)
    cash_statement_quarter = curs.fetchall()
    stock_code = cash_statement_quarter[len(cash_statement_quarter) - 1][0]
    period = cash_statement_quarter[len(cash_statement_quarter) - 1][1]
    conn.close()

    cash_statement_trailing = []
    for j in range(len(cash_statement_quarter[len(cash_statement_quarter) - 1])):
        if j == 0 or j == 1:
            continue
        cash_statement_trailing.append(cash_statement_quarter[len(cash_statement_quarter) - 1][j])

    for j in range(len(cash_statement_quarter) - 1):
        for k in range(len(cash_statement_quarter[j])):
            if k == 0 or k == 1:
                continue
            cash_statement_trailing[k - 2] += cash_statement_quarter[j][k]
    
    conn = pymysql.connect(host='localhost', user='root', password='root', db='mhdb', charset='utf8')
    curs = conn.cursor()
    
    sql = f"REPLACE INTO cash_statement_trailing VALUES ('{stock_code}', '{period}', '{cash_statement_trailing[0]}','{cash_statement_trailing[1]}','{cash_statement_trailing[2]}','{cash_statement_trailing[3]}','{cash_statement_trailing[4]}','{cash_statement_trailing[5]}','{cash_statement_trailing[6]}','{cash_statement_trailing[7]}','{cash_statement_trailing[8]}','{cash_statement_trailing[9]}', '{cash_statement_trailing[10]}', '{cash_statement_trailing[11]}','{cash_statement_trailing[12]}','{cash_statement_trailing[13]}','{cash_statement_trailing[14]}','{cash_statement_trailing[15]}','{cash_statement_trailing[16]}','{cash_statement_trailing[17]}','{cash_statement_trailing[18]}', '{cash_statement_trailing[19]}', '{cash_statement_trailing[20]}', '{cash_statement_trailing[21]}')"
    curs.execute(sql)

    conn.commit()
    conn.close()

In [6]:
conn = pymysql.connect(host='localhost', user='root', password='root', db='mhdb', charset='utf8')
curs = conn.cursor()
sql = f"SELECT DISTINCT stock_code FROM income_statement_quarter"
curs.execute(sql)
stock_code = curs.fetchall()

conn.close()

symbol_code = []
for i in range(len(stock_code)):
    symbol_code.append(stock_code[i][0])

In [7]:
len(symbol_code)

1783

In [11]:
for i in range(len(symbol_code)):
    conn = pymysql.connect(host='localhost', user='root', password='root', db='mhdb', charset='utf8')
    curs = conn.cursor()
    stock_code = symbol_code[i]
    sql = f"SELECT * FROM income_statement_quarter WHERE stock_code = '{stock_code}'"
    curs.execute(sql)
    income_statement_quarter = curs.fetchall()
    stock_code = income_statement_quarter[len(income_statement_quarter) - 1][0]
    period = income_statement_quarter[len(income_statement_quarter) - 1][1]
    conn.close()

    income_statement_trailing = []
    for j in range(len(income_statement_quarter[len(income_statement_quarter) - 1])):
        if j == 0 or j == 1:
            continue
        income_statement_trailing.append(income_statement_quarter[len(income_statement_quarter) - 1][j])

    for j in range(len(income_statement_quarter) - 1):
        for k in range(len(income_statement_quarter[j])):
            if k == 0 or k == 1:
                continue
            if k == 8 or k == 12:
                income_statement_trailing[k - 2] = 0
            else:
                income_statement_trailing[k - 2] += income_statement_quarter[j][k]
    
    conn = pymysql.connect(host='localhost', user='root', password='root', db='mhdb', charset='utf8')
    curs = conn.cursor()
    
    sql = f"REPLACE INTO income_statement_trailing VALUES ('{stock_code}', '{period}', '{income_statement_trailing[0]}','{income_statement_trailing[1]}','{income_statement_trailing[2]}','{income_statement_trailing[3]}','{income_statement_trailing[4]}','{income_statement_trailing[5]}','{income_statement_trailing[6]}','{income_statement_trailing[7]}','{income_statement_trailing[8]}','{income_statement_trailing[9]}', '{income_statement_trailing[10]}', '{income_statement_trailing[11]}','{income_statement_trailing[12]}','{income_statement_trailing[13]}','{income_statement_trailing[14]}','{income_statement_trailing[15]}','{income_statement_trailing[16]}','{income_statement_trailing[17]}')"
    curs.execute(sql)

    conn.commit()
    conn.close()